In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
import my_utils as mu

#!pip install phyaat  # if not installed yet
import phyaat
import phyaat as ph

PhyAAt Processing lib Loaded...


In [3]:
# Download dataset of one subject only (subject=1)
# To download data of all the subjects use subject =-1 or for specify for one e.g.subject=10

# Download original PhyAAt dataset
dirPath = ph.download_data(baseDir='code/PhyAAt_Data', subject=-1,verbose=0,overwrite=False)
baseDir='code/PhyAAt_Data'   # or dirPath return path from above

#returns a dictionary containing file names of all the subjects available in baseDir
SubID = ph.ReadFilesPath(baseDir)

#check files of subject=1
print(SubID)

Total Subjects :  0
{}


## highpass fitering and artifact removal

In [92]:
# highpass fitering and artifact removal
for i in sorted(SubID.keys()):
    print ('Now Loading Subject:', i)
    Subj = ph.Subject(SubID[i])
    eeg_before = Subj.getEEG()
    
    print('before shape: {} \n'.format(eeg_before.shape))
    #filtering with highpass filter of cutoff frequency 0.5Hz
    
    Subj.filter_EEG(band =[0.5],btype='highpass',order=5)
    Subj.correct(method='ICA',verbose=1,winsize=128*10)
    eeg_after = Subj.getEEG()
    
    print('after shape: {} \n'.format(eeg_after.shape))
    eeg_after.to_csv('eeg_clean/{}.csv'.format(i), index = False)
    
    eeg_check = pd.read_csv('eeg_clean/{}.csv'.format(i))
    print('read shape: ',eeg_check.shape)

Now Loading Subject: 1
before shape: (277548, 14) 

ICA Artifact Removal : extended-infomax
100%|####################################################################################################|
after shape: (277548, 14) 

read shape:  (277548, 14)
Now Loading Subject: 2
before shape: (282484, 14) 

ICA Artifact Removal : extended-infomax
100%|####################################################################################################|
after shape: (282484, 14) 

read shape:  (282484, 14)
Now Loading Subject: 3
before shape: (219064, 14) 

ICA Artifact Removal : extended-infomax
100%|####################################################################################################|
after shape: (219064, 14) 

read shape:  (219064, 14)
Now Loading Subject: 4
before shape: (351552, 14) 

ICA Artifact Removal : extended-infomax
100%|####################################################################################################|
after shape: (351552, 14) 

read shape:  (

## Extract feature from preprocessed EEG with different window size

In [3]:
filename = {
    1:'T1',
    2:'T2',
    3:'T3',

}

winsize = {
    '3s' : 128 * 3,
#     '2s' : 128 * 2,
#     '1s' : 128,
#     '0.5s' : 64,
    # '0.25s': 32
}

In [ ]:
# Extract feature from preprocessed EEG with different window size
for task_No in filename.keys():
    for win_length in winsize.keys():
        print('Now Extracting data for task{}_{}'.format(task_No, win_length))

        X_train = np.empty((0,84))
        y_train = np.empty((0,))
        X_test = np.empty((0,84))
        y_test = np.empty((0,))

        for i in sorted(SubID.keys()):
            print ('Now Loading Subject:', i)

            Subj = ph.Subject(SubID[i])
            eeg_clean = pd.read_csv('eeg_clean/{}.csv'.format(i))
            Subj.filter_EEG(band =[0.5],btype='highpass',order=5)
            Subj.updateEEG(eeg_clean)

            # Extract Rhythmic features for task=
            Subj_X_train,Subj_y_train,Subj_X_test, Subj_y_test = Subj.getXy_eeg(task=task_No, redo = True, winsize = winsize[win_length])
            print(Subj_X_train.shape,Subj_y_train.shape,Subj_X_test.shape, Subj_y_test.shape)
            
            X_train = np.vstack((X_train, Subj_X_train))
            y_train = np.hstack((y_train, Subj_y_train))
            X_test = np.vstack((X_test, Subj_X_test))
            y_test = np.hstack((y_test, Subj_y_test))

        print('DataShape: ',X_train.shape,y_train.shape,X_test.shape, y_test.shape)

        pd.DataFrame(X_train).to_csv('extracted_data/T{}_{}_X_train.csv'.format(task_No, win_length), index = False)
        pd.DataFrame(y_train).to_csv('extracted_data/T{}_{}_y_train.csv'.format(task_No, win_length), index = False)
        pd.DataFrame(X_test).to_csv('extracted_data/T{}_{}_X_test.csv'.format(task_No, win_length), index = False)
        pd.DataFrame(y_test).to_csv('extracted_data/T{}_{}_y_test.csv'.format(task_No, win_length), index = False)